# Optimización de hiperparámetros


Los hiperparámetros son piezas claves en los modelos de machine learning que nos permiten afinar el desempeño de dichos modelos y ajustarlos a los problemas que queremos resolver. 

Por lo tanto, ser capaces de optimizar sus valores y encontrar la mejor combinación de valores para los hiperparámetros de nuestros modelos es una tarea clave para poder maximizar el rendimiento que obtengamos con nuestros modelos. 

Necesitamos conocer, hasta cierto punto, la implicación que tiene cada hiperparámetro en cada modelo y sus posible valores. Entender en profundidad el significados y los posibles valores que pueden tener cada hiperparámetro de cada modelo nos permite obtener los mejores resultados de los modelos con los que vamos a trabajar, pero este módulo del bootcamp no espera llegar tan profundamente y eso es materia de estudio para otros cursos más especializados. En este módulo vamos a ver algunas de las técnicas más utilizadas para la optimización de los hiperparámetros de los modelos con los que vamos a trabajar. 

La opción de poder ir modificando manualmente los valores de los hiperparámetros de nuestros modelos y comparar los resultados que nos ofrecen existe, pero es evidente que un ajuste de este tipo suele llevar mucho tiempo, no siempre se realiza de forma rigurosa y dificulta inmensamente la sistematización de los resultados de nuestros modelos. 

Otra opción que tenemos disponible es aplicar técnicas automatizadas y sencillas como son **Grid Search** y **Random Search** que suelen obtener buenos resultados. A continuación iremos viendo las características de ambas técnicas, aunque ambas técnicas presentan un elevado coste de tiempo y recursos computacionales. 

Finalmente, podemos encontrar el valor optimo para nuestros hiperparámetros utilizando la técnica de **optimización bayesiana**. A pesar de que presenta una mayor dificultad de implementación, esta técncia nos permite obtener buenos resultados con un menor coste en tiempo y recursos computacionales. 


## GridSearch

La técnica de GridSearch funciona probando todas las posibles combinaciones de parámetros que se desean probar en el modelo. Cada uno de esos parámetros se prueba en una serie de iteraciones de validación cruzada. Finalmente, el modelo que presente los mejores resultados es seleccionado y sus hiperparámetros son accesibles para que podamos utilizar dicho modelo y podamos evaluar cuáles han sido los mejores hiperparámetros que podemos utilizar con nuestro modelo. 

La librería de scikit-learn nos proporciona una implementación que nos permite aplicar la técnica de GridSearch con nuestros modelos para poder obtener de forma fácil los hiperparámetros optimos para nuestros problemas. 

Buscando en la documentación de scikit-learn del modelo o modelos para los que queremos encontrar los hiperparámetros óptimos, debemos seleccionar qué hiperparámetros son los que vamos a optimizar, todos o un subconjunto de los hiperparámetros disponibles del modelo. 

Una vez hemos decidido qué hiperparámetros vamos a optimizar, tenemos que crear un diccionario con el nombre del hiperparámetro como clave y un array de valores que queremos probar para cada hiperparámetro como valor del diccionario. Este es el elemento clave que nos permite ir probando distintas combinaciones de hiperparámetros. Cuantos más valores añadamos para probar, mayor será el número de combinaciones disponibles y por lo tanto mayor será el tiempo que se necesite para la obtención de la mejor combinación de hiperparámetros. 

Vamos a ver un ejemplo de aplicación de GridSearch: 



In [26]:
import os
import numpy as np
import pandas as pd  
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [12]:
# vamos a cargar los datos que vamos a utilizar, del set de datos relacionados con el Titanic
df = pd.read_csv('train_titanic.csv', index_col=0)

# Separamos nuestros datos entre la columna objetivo y el resto de columnas que utilizaremos en nuestro modelo
y= df.Survived
features = df.drop(columns=['Survived'],axis=1)

In [13]:
# Vamos a preparar rápidamente nuestros datos para poder utilizarlos en el modelo
# vamos a eliminar las columnas que no nos interesen
features = features.drop(columns=['Cabin','Name','Ticket'], axis=1)

# Transformamos las variables categóricas en numéricas 
X = pd.get_dummies(features)

# Rellenamos los valores vacíos con un valor de cero
X = X.fillna(0)

In [15]:
# Vamos a crear una instancia de nuestro modelo, en este caso el random forest 
# Optamos por los hiperparámetros por defecto para poder comparar los resultados que nos ofrece
# el modelo con los valores óptimos que encontremos de nuestros hiperparámetros
rf = RandomForestClassifier()

In [16]:
# Separamos nuestros datos en set entrenamiento y de pruebas siguiendo un enfoque de 
# validación simple
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [17]:
# entrenamos nuestro modelo con los datos del set de entrenamiento
rf.fit(X_train, y_train)

RandomForestClassifier()

In [19]:
# Y calculamos la precisión de clasificación del modelo 
precision_defecto = rf.score(X_test, y_test)
precision_defecto

0.8156424581005587

In [20]:
# ahora vamos a empezar a crear nuestro diccionario de hiperparámetros que vamos
# a probar para buscar la mejor combinación. 

n_estimators = [10, 100, 1000]
max_features = ['sqrt','log2']
criterion = ['gini', 'entropy','log_loss']

rf_grid = {'n_estimators':n_estimators, 'max_features':max_features, 'criterion':criterion}

Podemos encontrar la lista completa de los hiperparámetros del modelo de random forest en la documentación oficial de scikit-learn en el siguiente [link](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

In [22]:
# vamos a crear un objeto que nos permita aplicar la técnica de grid search para nuestro modelo
grid_search = GridSearchCV(estimator=rf, param_grid=rf_grid)

In [27]:
# el resultado de GridSearch nos da el modelo optimo y por lo tanto podemos entrenarlo
# y obtener el valor de precisión de clasificación de nuestro modelo con hiperparámetros óptimos

grid_search_rf = grid_search.fit(X_train,y_train)

In [28]:
# Una vez ha encontrado los mejore valores para los hiperparámetros de nuestro modelo
# podemos ver esos valores

grid_search_rf.best_params_

{'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 10}

In [30]:
# Calculamos la precisión de clasificación de nuestro modelo con los hiperparámetros óptimos
precision_gridsearch = grid_search_rf.score(X_test,y_test)
precision_gridsearch

0.8100558659217877

La precisión del modelo se ve modificada al variar los valores de los hiperparámetros. Podemos seguir explorando con otros valores y otros hiperparámetros para conseguir una mejora en la precisión del modelo. 

## RandomSearch

Dado que la aplicación de la técnica de GridSearch puede consumir grandes cantidades de recursos computacionales dependiendo del tamaño del set de datos y el número de combinaciones que queremos realizar, los científicos de datos buscaron otra alternativa que fuese más rápida. Trabajaron en la técnica de RandomSearch, que se basa en muestrear al azar a partir de una varieddad de parámetros los valores que queremos dar a los hiperparámetros de nuestro modelo. La idea principal detrás de está técnica es que se puede cubrir el conjunto de parámetros óptimos más rápido que utilizando el GridSearch. Esta técnica,sin embargo, es considerada ingenua (naive), puesto que no sabe ni recuerdad nada de sus ejecuciones anteriores. 

La librería de scikit-learn nos proporciona una implementación que nos permite aplicar la técnica de RandomSearch con nuestros modelos para poder obtener de forma fácil los hiperparámetros optimos para nuestros problemas.

Al igual que con la técnica de GridSearch, debemos crear un diccionario con los valores que queremos poder probar para cada uno de los hiperparámetros que queremos optimizar y luego utilizar dicho diccionario en el proceso de optimización. 

Vamos a ver, siguiendo con el ejemplo anterior, como aplicar RandomSearch: 

In [31]:
from sklearn.model_selection import RandomizedSearchCV

In [32]:
# creamos el objeto que nos permite aplicar la técnica de RandomSearch
random_search = RandomizedSearchCV(rf, param_distributions=rf_grid)

In [33]:
# Aplicamos la técnica de RandomSearch para conseguir los valores óptimos de los 
# hiperparámetros para nuestro modelo y entrenamos el mejor modelo con nuestros datos

random_search_rf = random_search.fit(X_train,y_train)

In [34]:
# Una vez ha encontrado los mejore valores para los hiperparámetros de nuestro modelo
# podemos ver esos valores

random_search_rf.best_params_

{'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 10}

In [35]:
# Calculamos la precisión de clasificación de nuestro modelo con los hiperparámetros óptimos
precision_randomsearch = random_search_rf.score(X_test,y_test)
precision_randomsearch

0.770949720670391

En este caso, vemos que la precisión del modelo se ha visto afectada por la aplicación de la técnica de RandomSearch, pero siempre podemos modificar el diccionario de valores que queremos probar para cada hiperparámetro y añadir otros hiperparámetros para poder conseguir un mejor rendimiento de nuestro modelo. 

## Optimización bayesiana de hiperparámetros

La técnica de optimización bayesiana se basa en buscar el valor minimo para una función objetivo, construyendo a su vez una distribución de probabilidades de los valores de esa función. La optimización de esa distribución de probabilidad es más fácil a nivel computacional que la optimización de la función original. 

La diferencia entre la optimización bayesiana y las técnicas como GridSearch y RandomSearch es que la optimización bayesiana utiliza los resultados del pasado para seleccionar cuales van a ser los siguientes valores a evaluar. 

En la optimización bayesiana de hiperparámetros de modelos de machine learning, la función objetivo es el error de validación del modelo utilizando un set de hiperparámetros. El objetivo de esta técnica es encontrar los valores de los hiperparámetros que generan un valor mínimo del error de validación, esperando que esos hiperparámetros generen también un valor mínimo del error en el set de pruebas. 

Evaluar la función objetivo es computacionalmente caro puesto que necesitas entrenar el modelo con un set de hiperparámetros. Idealmente, queremos una técnica que nos permite explorar el espacio de valores mientras que limitamos el número de evaluaciones del error de validación con malas combinaciones de hiperparámetros. La técnica de optimización bayesiana utiliza una distribución de probabilidades que se "concentra" en los hiperparámetros que presentan mejores perspectivas basándose en los resultados de las evaluaciones pasadas. 

Vamos a seguir con el ejemplo anterior, utilizando ahora la técnica de optimización bayesiana de los hiperparámetros de nuestro modelo

In [43]:
import csv
from hyperopt import STATUS_OK, tpe, hp, Trials
from hyperopt.fmin import fmin
from timeit import default_timer as timer
from sklearn.metrics import mean_squared_error

In [46]:
MAX_EVALS = 500
N_FOLDS= 10
seed = 123

In [47]:
# Definimos nuestra función objetivo que buscará la optimización de nuestro modelo de 
# random forest utilizando los hiperparámetros que queremos optimizar
def objective(params):
    est=int(params['n_estimators'])
    md=int(params['max_depth'])
    msl=int(params['min_samples_leaf'])
    mss=int(params['min_samples_split'])
    model=RandomForestClassifier(n_estimators=est,max_depth=md,min_samples_leaf=msl,min_samples_split=mss)
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    score=mean_squared_error(y_test,pred)
    return score

In [48]:
# Esta función nos permite definir el espacio de los valores que pueden tomar los hiperparámetros 
# que vamos a optimizar. Además cuando escoge los valores que vamos a probar para nuestros hiperparámetros
# calcula el valor de la función objetivo y busca el valor óptimo para minimizar el resultado de esa 
# función objetivo
def optimize(trial):
    params={'n_estimators':hp.uniform('n_estimators',100,500),
           'max_depth':hp.uniform('max_depth',5,20),
           'min_samples_leaf':hp.uniform('min_samples_leaf',1,5),
           'min_samples_split':hp.uniform('min_samples_split',2,6)}
    best=fmin(fn=objective,space=params,algo=tpe.suggest,trials=trial,max_evals=500,rstate=np.random.RandomState(seed))
    return best

In [49]:
#Finalmente lanzamos el proceso de optimización.
trial=Trials()
best=optimize(trial)

100%|██████████| 500/500 [05:32<00:00,  1.51it/s, best loss: 0.12849162011173185]


In [50]:
# podemos ver cuales son los mejores valores de los hiperparámetros que hemos conseguido con el proceso de 
# optimización bayesiana
print(best)

{'max_depth': 8.979353900393882, 'min_samples_leaf': 1.5188500441640422, 'min_samples_split': 2.4685823620647405, 'n_estimators': 102.79481832266357}


In [57]:
# Creamos el modelo con los valores óptimos para los hiperparámetros que hemos encontrado
# tomamos los valores enteros más cercanos a los valores obtenidos puesto que los hiperparámetros
# esperan tener valores enteros
rf_bayesiano = RandomForestClassifier(max_depth=9,
                                      min_samples_leaf=2, 
                                      min_samples_split=3,
                                      n_estimators=103)

In [58]:
#entrenamos el modelo con el set de entrenamiento
rf_bayesiano.fit(X_train, y_train)

RandomForestClassifier(max_depth=9, min_samples_leaf=2, min_samples_split=3,
                       n_estimators=103)

In [60]:
# calculamos la precisión de clasificación del modelo óptimo
precision_bayesiano = rf_bayesiano.score(X_test, y_test)
precision_bayesiano

0.8659217877094972